In [23]:
!python3.10 -m pip install langchain==0.2.9 langchain-ollama langserve==0.2.3


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip3.10 install --upgrade pip


In [24]:
from langchain_ollama import OllamaLLM

model = OllamaLLM(model="llama3.2")

In [25]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="Translate the following from English into Italian"),
    HumanMessage(content="hi!"),
]

model.invoke(messages)

'Ciao! (CHOW)'

In [26]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

result = model.invoke(messages)

parser.invoke(result)

'"Ciao!"'

In [27]:
# Simple formatter function
def simple_formatter(parsed_data):
    """
    This function formats the parsed data by adding a label.
    
    parsed_data: The parsed output (assumed to be a string from the previous step)
    """
    return f"Formatted Result: {parsed_data}"

In [28]:
chain = model | parser | simple_formatter
chain.invoke(messages)

'Formatted Result: Ciao! (CHOW)'

In [29]:
from langchain_core.prompts import ChatPromptTemplate

system_template = "Translate the following into {language}:"

prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)

result = prompt_template.invoke({"language": "Mandarin", "text": "hi"})

result

result.to_messages()

[SystemMessage(content='Translate the following into Mandarin:'),
 HumanMessage(content='hi')]

In [30]:
chain = prompt_template | model | parser
chain.invoke({"language": "Mandarin", "text": "hi"})

'The translation of "hi" to Mandarin is "nǐ hǎo". \n\nNote that in Mandarin, it\'s more common to use "" (xiè xiè) for a friendly greeting or acknowledgement.'

In [33]:
# Client for serve.py
from langserve import RemoteRunnable

remote_chain = RemoteRunnable("http://localhost:8000/chain/")
remote_chain.invoke({"language": "italian", "text": "hi"})

'The translation of "hi" in Italian is "ciao". However, depending on the context and level of formality, you might also use:\n\n* "buongiorno" (good morning)\n* "buona sera" (good evening)\n* "giovanni" (hello)'